In [1]:
import os, tempfile
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model


#from matplotlib.animation import FuncAnimation



/home/roberta/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/roberta/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/roberta/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/roberta/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
model10 = load_model("whistle3.hd5")
modelT = load_model("whistle5.hd5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [3]:
import sounddevice as sd
import queue

chunk_length = (300+350) * 64 + 1024
pre_trigger = 110
chunk = np.zeros((chunk_length,1),dtype='float32')
chunk[30:40,0] = 10.0
recorded_length = 0;

triggered = False
captured = False
threshold = 0.3

q = queue.Queue()




def callback(indata, frames, time, status):
    global triggered,threshold
    if status:
        print(status)
    q.put(indata[::1, [0]])    
    if ~triggered:    
        maxi = indata.max()
        if maxi>threshold:
            triggered = True

def catch_sound(hits = 10,trigger_threshold = 0.071):
    global chunk,recorded_length,triggered,threshold
    threshold = trigger_threshold
    
    max_hits = hits
    stride = 64 # int(64 * 48000/44100) 

    recorded_length = 0
    triggered = False
    instream =  sd.InputStream(channels=1, callback=callback, blocksize=1024,dtype='float32')     
    with instream:
        while True:
            item = q.get()
            if item is None:
                continue
            ilen = len(item)
            chunk = np.roll(chunk, -ilen, axis=0)
            chunk[-ilen:] = item
            if triggered:
                recorded_length +=ilen
            if recorded_length > chunk_length-pre_trigger:
                triggered = False
                print("..")
                f, t, Sxx = signal.spectrogram(chunk[:,0], 44100.,nperseg = 512,noverlap = 512 - stride)
                St = Sxx[0:100,0:250]
                lsx = St/np.mean(St)
        
                imlru = np.reshape(lsx,(1,100,250,1))
  
                p1 =  model10.predict(imlru)
                p2 = modelT.predict(np.reshape(lsx[20:50,:],(1,30,250,1)))

                print(np.argmax(p1),p1[0,np.argmax(p1)])
            
                print(np.argmax(p2),p2[0,np.argmax(p2)])
                
                sd.play(chunk[:,0])
                recorded_length = 0
                max_hits -=1
                if max_hits == 0:
                    break
                #break
            q.task_done()

In [4]:
catch_sound()

..
1 0.9964915
3 0.99794286
..
5 0.55394465
1 0.96095747
..
0 0.7574239
1 0.939205
..
5 0.7819565
6 0.99881965
..
3 0.7662468
1 1.0
..
3 0.56327873
1 0.99992836
..
5 0.92505
0 0.9736501
..
1 0.89425164
1 0.9999989
..
3 0.78574806
1 0.41625315
..
5 0.96723324
1 0.9999938
